# PySpark Lab

In [1]:

from sparknlp.base import Pipeline
from sparknlp.annotator import DocumentAssembler, LanguageDetectorDL
import pyspark
from pyspark.sql.functions import  col, from_unixtime
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, DoubleType, LongType
from appinsight.infrastructure.io import IOService
pyspark.__version__


'3.5.0'

## Spark Session

In [2]:
spark = (
    SparkSession.builder.appName("AppInsight")
    .master("local[*]")
    .config("spark.driver.memory", "32g")
    .config("spark.executor.memory", "32g")
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .config("spark.kryoserializer.buffer.max", "2000M")
    .config("spark.driver.maxResultSize", "0")
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.3.3")
    .config("spark.sql.legacy.parquet.nanosAsLong", "true")    
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/05/24 07:05:58 WARN Utils: Your hostname, Leviathan resolves to a loopback address: 127.0.1.1; using 172.29.219.103 instead (on interface eth0)
24/05/24 07:05:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/john/anaconda3/envs/appinsight/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/john/.ivy2/cache
The jars for the packages stored in: /home/john/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-923465b1-16f9-45e1-aee3-3d236e86c992;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.3.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in central
	f

## Data

In [3]:
schema = StructType([ \
    StructField("id", StringType(), True), \
    StructField("app_id", StringType(), True), \
    StructField("app_name", StringType(), True), \
    StructField("category_id", StringType(), True), \
    StructField("category", StringType(), True), \
    StructField("author", StringType(), True), \
    StructField("rating", DoubleType(), True), \
    StructField("title", StringType(), True), \
    StructField("content", StringType(), True), \
    StructField("review_length", LongType(), True), \
    StructField("vote_count", LongType(), True), \
    StructField("vote_sum", LongType(), True), \
    StructField("date", LongType(), True), \
    
    ])

In [4]:
fp1 = "data/dev/00_raw/reviews.pkl"
fp2 = "data/dev/00_raw/reviews.parquet"
df1 = IOService.read(filepath=fp1)
df1.to_parquet(fp2)


## Create Spark DataFrame

In [5]:
df2 = spark.read.option("header", "true").schema(schema).parquet(fp2)
df2 = df2.withColumn("date", from_unixtime(col("date")/1000000000).cast('timestamp'))
df2.printSchema()
df2.take(5)

root
 |-- id: string (nullable = true)
 |-- app_id: string (nullable = true)
 |-- app_name: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- author: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- title: string (nullable = true)
 |-- content: string (nullable = true)
 |-- review_length: long (nullable = true)
 |-- vote_count: long (nullable = true)
 |-- vote_sum: long (nullable = true)
 |-- date: timestamp (nullable = true)



[Row(id='1147487073', app_id='361309726', app_name='Pages', category_id='6007', category='Productivity', author='Ali Boorghani', rating=4.0, title='ترك', content='اخه اپ استور ترك تازه يه ساعته دانلود كردم برا چي براش اپديت اومده؟؟؟؟😳😡', review_length=14, vote_count=0, vote_sum=0, date=datetime.datetime(2015, 2, 13, 16, 13)),
 Row(id='6725734679', app_id='1377084545', app_name='EMOJI Face Recorder', category_id='6002', category='Utilities', author='Is a blast', rating=1.0, title='I HATE THIS GAME!!!🤮🤮🤮', content='I hate this game don't download it u can't even stick ur tongue it doesn't even copy ur face u can't even record the is a liar they lie to u you soo much don't get ittttt!!!', review_length=37, vote_count=0, vote_sum=0, date=datetime.datetime(2020, 12, 6, 22, 52, 2)),
 Row(id='7007544886', app_id='764595159', app_name='Perfect Hairstyle:New Hair Cut', category_id='6012', category='Lifestyle', author='DRghpl56', rating=5.0, title='DRghpl56', content='Great app had fun with styl

## Pipeline

In [6]:
documentAssembler = DocumentAssembler().setInputCol("content").setOutputCol('document')

#language detection
languageDetector = LanguageDetectorDL.pretrained() \
  .setInputCols(["document"]) \
  .setOutputCol("language")

pipe = Pipeline(stages=[documentAssembler, languageDetector])
results = pipe.fit(df2).transform(df2)

ld_wiki_tatoeba_cnn_21 download started this may take some time.
Approximate size to download 7.1 MB
[ | ]ld_wiki_tatoeba_cnn_21 download started this may take some time.
Approximate size to download 7.1 MB
Download done! Loading the resource.
[ / ]

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/john/anaconda3/envs/appinsight/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/john/anaconda3/envs/appinsight/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/john/anaconda3/envs/appinsight/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


#
# A fatal error has been detected by the Java Runtime Environment:
#
#  SIGILL (0x4) at pc=0x00007f0afb672260, pid=22624, tid=0x00007f0c2d96b640
#
# JRE version: OpenJDK Runtime Environment (8.0_402-b06) (build 1.8.0_402-8u402-ga-2ubuntu1~22.04-b06)
# Java VM: OpenJDK 64-Bit Server VM (25.402-b06 mixed mode linux-amd64 )
# Problematic frame:
# C  [libtensorflow_framework.so.2+0x16bc260]  nsync::nsync_mu_init(nsync::nsync_mu_s_*)+0x0
#
# Failed to write core dump. Core dumps have been disabled. To enable core dumping, try "ulimit -c unlimited" before starting Java again
#
# An error report file with more information is saved as:
# /home/john/projects/appinsight/hs_err_pid22624.log
#
# If you would like to submit a bug report, please visit:
#   http://bugreport.java.com/bugreport/crash.jsp
# The crash happened outside the Java Virtual Machine in native code.
# See problematic frame for where to report the bug.
#


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/john/anaconda3/envs/appinsight/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/john/anaconda3/envs/appinsight/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/john/anaconda3/envs/appinsight/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


[OK!]


Py4JError: An error occurred while calling z:com.johnsnowlabs.nlp.pretrained.PythonResourceDownloader.downloadModel

## Resullts

In [ ]:
results.select("language.result").show()